<a href="https://colab.research.google.com/github/lpoggetto/fiap_recommendation_system/blob/main/trabalho_SR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
from pandas import json_normalize
from time import sleep

# URL da API Open Finance (substitua com a URL da sua API)
urls = [
'https://api.itau/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://opendata.api.nubank.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards','https://resources.openbanking.picpay.com/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://openbanking.api.santander.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
#'https://openbanking.bib.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://api.mercadopago.com/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://opendata.api.bb.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://api-bmg.bancobmg.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
#'https://api.bradesco.com/next/open-banking/opendata-creditcards/v1/personal-credit-cards',
# 'https://api.bradesco.com/bradesco/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://openbanking.banrisul.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://api-openbanking.bancopan.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://api.safra.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://banking-openfinance.xpi.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://banking-openfinance.xpi.com.br/rico/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://banking-openfinance-azimutbrasil.xpi.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://banking-openfinance.xpi.com.br/clear/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://banking-openfinance.xpi.com.br/modal/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://banking-openfinance-montebravo.xpi.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://banking-openfinance-whg.xpi.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://api-openbanking.bvopen.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://api.sicredi.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://api-opf.asa.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://ob-api.sisprimedobrasil.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
# 'https://openfinance.sicoob.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://authorization.openapi.sofisa.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
# 'https://api.openbanking.caixa.gov.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
# 'https://api.bradesco.com/bradescard/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://api.e-unicred.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://apps.neon.com.br/open-finance/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://openbanking.digio.com.br/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://btgmais.openbanking.btgpactual.com/open-banking/opendata-creditcards/v1/personal-credit-cards',
'https://api.openfinance.bnb.gov.br/open-banking/opendata-creditcards/v1/personal-credit-cards'
]

# Fazendo a requisição GET à API
for url in urls:
  response = requests.get(url)

  # Verificando se a requisição foi bem-sucedida (status code 200)
  if response.status_code == 200:
    try:
      df1 = pd.json_normalize(response.json(), 'data')
      dffinal = pd.concat([dffinal, df1])
    except:
      dffinal = pd.json_normalize(response.json(), 'data')
  else:
      print(f"Falha na requisição. Código de status: {response.status_code}")


Falha na requisição. Código de status: 403


In [24]:
def recommend_cards(dataset, network=None, has_rewards=None, brand=None, product_type=None):
    """
    Filter credit cards based on user criteria

    Parameters:
    - dataset: Your DataFrame (dffinal)
    - network: Credit card network (VISA, Mastercard, American Express, etc.)
    - has_rewards: Boolean for whether card has rewards program (True/False)
    - brand: Issuer brand name
    - product_type: Type of product (Credit Card, Charge Card, etc.)

    Returns:
    - Filtered DataFrame with recommended cards
    """

    # Instead of using query, directly filter using boolean indexing:
    filtered_dataset = dataset

    if network:
        filtered_dataset = filtered_dataset[filtered_dataset['identification.creditCard.network'] == network]
    if has_rewards is not None:
        filtered_dataset = filtered_dataset[filtered_dataset['rewardsProgram.hasRewardProgram'] == has_rewards]
    if brand:
        filtered_dataset = filtered_dataset[filtered_dataset['participant.brand'] == brand]
    if product_type:
        filtered_dataset = filtered_dataset[filtered_dataset['identification.product.type'] == product_type]

    return filtered_dataset[['participant.brand', 'participant.name', 'identification.product.type', 'identification.creditCard.network', 'rewardsProgram.hasRewardProgram']]

In [25]:
visa_rewards = recommend_cards(dffinal, network='VISA', has_rewards=True)

In [26]:
visa_rewards

,participant.brand,participant.name,identification.product.type,identification.creditCard.network,rewardsProgram.hasRewardProgram
20,ITAU,BANCO ITAUCARD,CLASSIC_INTERNACIONAL,VISA,True
21,ITAU,BANCO ITAUCARD,PLATINUM,VISA,True
22,ITAU,BANCO ITAUCARD,PLATINUM,VISA,True
4,Banco Santander,BCO SANTANDER (BRASIL) S.A.,GOLD,VISA,True
5,Banco Santander,BCO SANTANDER (BRASIL) S.A.,INFINITE,VISA,True
